In [ ]:
using Revise
using LilGuys
using CairoMakie, Arya
LP = LilGuys.Plots

In [ ]:
using Printf

In [ ]:
import DataFrames: DataFrame, rename!

In [ ]:
models_dir = "/astro/dboyea/dwarfs/analysis/isolation"

In [ ]:
function load_profile(name) 
    path = joinpath(models_dir, "$name/profiles.hdf5")
    profiles = LilGuys.read_structs_from_hdf5(path, LilGuys.MassProfile3D)
    idx = parse.(Int, first.(profiles))
    profiles = last.(profiles)
    return profiles[sortperm(idx)]
    
end

In [ ]:
t_max_host = 316 # computed with agama at peri=50.95 
M_max_host = 52.437
r_peri = 50.95
ρ_peri = M_max_host / (4π/3 * r_peri^3)

In [ ]:
halo = LilGuys.load_profile("$models_dir/1e7/fiducial/halo.toml")

In [ ]:
Base.@kwdef struct NFWZeno <: LilGuys.SphericalProfile
	m_a = LilGuys.A_NFW(1)
	a = 1
	b = 64
	taper = :exp
end


function calc_ρ_nfw(halo, r)
	return halo.m_a / (4π * LilGuys.A_NFW(1) * r * (halo.a + r)^2)
end

# ╔═╡ dd47be7c-99dc-4f66-8bb4-33f827438a26
function calc_ρ_e(halo, r)
	ρ_b = calc_ρ_nfw(halo, halo.b)

	b = halo.b
	γ = b/(b+halo.a) - 1/2
	return ρ_b * (b/r)^2 * exp(-2γ * (r/b - 1))
end

# ╔═╡ 4cae6065-6c19-4667-b5ae-87ed49958e70
function LilGuys.calc_ρ(halo::NFWZeno, r)
	if r < halo.b
		return calc_ρ_nfw(halo, r)
	else
		return calc_ρ_e(halo, r)
	end
end

# Plotting functions

In [ ]:
function compare_densities(profiles; errskip=1)
    fig = Figure()
    ax = LP.rho_axis(fig[1, 1])
    ax.limits=(-2, 3, -12, 0)


    for i in eachindex(profiles)
        label, profs = profiles[i]
        profile = profs[1]
        lines!(profile.log_r, log10.(profile.rho),
            linestyle=:dot,
            color=COLORS[i]
        )
    
        profile = profs[end]
        lines!(profile.log_r, log10.(profile.rho), 
            color=COLORS[i],
            label=label
        )    
    end
    log_r = LinRange(-2, 3, 1000)
    ρ = calc_ρ.(halo, 10 .^ log_r)
    lines!(log_r, log10.(ρ), color=:black, linestyle=:dash)
    
    
    axislegend()

    ax_res = LP.rho_axis(fig[2, 1])
    ax_res.ylabel = L"(\rho-\rho_\textrm{exp}) / \rho_\textrm{exp}"
    ax_res.limits = (-2, 3, -1, 1)

    for i in eachindex(profiles)
        profile = profiles[i].second[end]

        x = profile.log_r
        y_exp = calc_ρ.(halo, 10 .^ x)
        dy = profile.rho .- y_exp

        res = dy ./ y_exp
        res_err = profile.rho_err ./ y_exp
        lines!(x, res, color=COLORS[i])
        
        si = 1:errskip:length(res)
        errorbars!(x[si], res[si], res_err[si], color=COLORS[i])
    end
    hlines!(0, color=:black)


	linkxaxes!(ax, ax_res, )
	rowsize!(fig.layout, 2, Auto(0.3))
	hidexdecorations!(ax, grid=false)
    
    fig

end

In [ ]:
function compare_vcirc1(profile, halo=halo; res_limits=0.1)
    xlims = (-2, 3)
    
    fig = Figure()
    ax = Axis(fig[1, 1],
        xlabel=LP.log_r_label,
        ylabel=L"$\log\;v_\textrm{circ}$ / km\,s$^{-1}$",
        limits=(xlims[1], xlims[2], 0, 1.6),
    )
    
    lines!(profile.log_r, log10.(profile.v_circ * V2KMS), label="model",
    )

    x = LinRange(xlims[1], xlims[2], 1000)
    y = calc_v_circ.(halo, 10 .^ x)
    lines!(x, log10.(y * V2KMS), linestyle=:dash, color=:black, label="analytic")

    axislegend(position=:rb)

    # residual

    ax_res = Axis(fig[2, 1],
        xlabel=LP.log_r_label,
        ylabel=L"\Delta\,v",
        limits=(xlims[1], xlims[2], -res_limits, res_limits),
    )
    

    x = profile.log_r_bins[2:end]
    y_exp = calc_v_circ.(halo, 10 .^ x)
    dy = profile.v_circ .- y_exp


    res = dy ./ y_exp
    res_err = profile.v_circ_err  ./ y_exp
    lines!(x, res)
    errorbars!(x, res, res_err)
    
    hlines!(0, color=:black)


	linkxaxes!(ax, ax_res, )
	rowsize!(fig.layout, 2, Auto(0.3))
	hidexdecorations!(ax, grid=false)
    
    fig
end

In [ ]:
function compare_vcirc(profiles; errskip=1)
    xlims = (-2, 3)
    
    fig = Figure()
    ax = Axis(fig[1, 1],
        xlabel=LP.log_r_label,
        ylabel=L"$\log\,v_\textrm{circ}$ / km\,s$^{-1}$",
        limits=(xlims[1], xlims[2], -0.1, 1.7),
        )
    pi = 1

    
    for i in eachindex(profiles)
        label, profs = profiles[i]
        profile = profs[1]
        lines!(profile.log_r, log10.(profile.v_circ * V2KMS),
            linestyle=:dot,
            color=COLORS[i]
        )
        
        profile = profs[end]
        lines!(profile.log_r, log10.(profile.v_circ* V2KMS), 
            color=COLORS[i],
            label=label
        )


    
    end

    x = LinRange(xlims[1], xlims[2], 1000)
    y = calc_v_circ.(halo, 10 .^ x)
    lines!(x, log10.(y * V2KMS), linestyle=:dash, color=:black, label="NFW")

    axislegend(position=:rb)

    # residual

    ax_res = Axis(fig[2, 1],
        xlabel=LP.log_r_label,
        ylabel=L"\Delta\,v\,/v_\textrm{exp}",
        limits=(xlims[1], xlims[2], -2, 2),
    )
    
    for i in eachindex(profiles)
        label, profs = profiles[i]
        profile = profs[end]

        x = profile.log_r_bins[2:end]
        y_exp = calc_v_circ.(halo, 10 .^ x)
        dy = profile.v_circ .- y_exp


        res = dy ./ y_exp
        res_err = profile.v_circ_err  * V2KMS
        scatterlines!(x, res, color=COLORS[i], markersize=3)

        idx = 1:errskip:length(res)
        errorbars!(x[idx], res[idx], res_err[idx], color=COLORS[i])
    end
    hlines!(0, color=:black)


	linkxaxes!(ax, ax_res, )
	rowsize!(fig.layout, 2, Auto(0.3))
	hidexdecorations!(ax, grid=false)
    
    fig
end

In [ ]:
function compare_t_dyn(profiles, halo=halo)
    fig = Figure()
    ax = Axis(fig[1, 1],
        xlabel=LP.log_r_label,
        ylabel=L"$\log\, t_\textrm{circ}$ / Gyr",
        limits=(-3, 3, -2, 0.5),
        )
    pi = 1
    
    for i in eachindex(profiles)
        
        #profile = profiles[i].second[1]
    
        label, profs = profiles[i]
        profile = profs[end]
        
        t = 2π * 10 .^ profile.log_r ./ (profile.v_circ) * T2GYR
        lines!(profile.log_r, log10.(t), 
            color=COLORS[i],
            label=label,
        )
    
    
    end
    

    if halo !== nothing
        log_r = LinRange(-3, 3, 1000)
        r = 10 .^ log_r
        t = @. 2π * r / LilGuys.calc_v_circ(halo, r) * T2GYR
        lines!(log_r, log10.(t), 
            color=:black,
            label = "analytic"
            )
    end
        
    axislegend()
        
    fig

end

In [ ]:
simulation_dir = "/astro/dboyea/dwarfs/simulations/isolation"

In [ ]:
function load_cpu_use(model_dir::String)
    filename = joinpath(simulation_dir, model_dir, "out/cpu.csv")

    lines = readlines(filename)
    lines[1] = replace(lines[1], "MULTIPLEDOMAIN, " => "")

    data = [split(line, ", ")[1:end-1] for line in lines[2:end]]
    
    columns = split(lines[1], ", ")[1:end-1]
    columns = replace.(columns, "1"=>"", "2"=>"_TOT", "3"=>"_PERCEN")

    Ncol = length(columns)

    data = [[row[i] for row in data] for i in 1:Ncol]
    df = DataFrame(data, Symbol.(columns))

    convert_types!(df)

    return df
end

In [ ]:
# Function to convert SubString values to Int, Float, or leave as String
function convert_types!(df::DataFrame)
    for col in names(df)
        df[!, col] = [tryparse(Int, val) !== nothing ? tryparse(Int, val) :
                      tryparse(Float64, val) !== nothing ? tryparse(Float64, val) :
                      val for val in df[!, col]]
    end
end

In [ ]:
function get_tot_cpu(model_dir::String)
    df = load_cpu_use(model_dir)

    return df.CPU_ALL_TOT[end] * df.CPUS[end] 
end

In [ ]:
function cpu_summary(model_dir::String)
    df = load_cpu_use(model_dir)

    tot = df.CPU_ALL_TOT[end] * df.CPUS[end] / df.TIME[end]
    percen_tree = LilGuys.mean(df.CPU_TREE_PERCEN)

    if "CPU_PM_GRAVITY_PERCEN" ∈ names(df)
        percen_tree = percen_tree + LilGuys.mean(df.CPU_PM_GRAVITY_PERCEN)
    end

    return tot, percen_tree
end

In [ ]:
function print_cpus_of_models(folders, names=folders; simplify_name=false)
    if simplify_name
        names = @. basename(names)
    end
    name_width = maximum(length.(names))
    
    @printf "%-*s\t%-8s\t%-5s\n" name_width "model" "cpu s/T0" "%grav"
    println("-"^name_width, "\t", "-"^8, "\t", "-"^8)

            
    
    for (folder, name) in zip(folders, names)
    
        try
            cpu, tree = cpu_summary(folder)

            
            @printf "%-*s\t%8.2f\t%4.1f" name_width name cpu tree
            println()
        catch e
        end
    end
end

In [ ]:
function print_cpus_in_folder(subfolder::String)
    folders = readdir(joinpath(models_dir, subfolder ))

    print_cpus_of_models(joinpath.([subfolder], folders), simplify_name=true)
   
end

## Orbit Profiles

In [ ]:
model_names = [
    "V32_r5",
    "V32_r2.4",
   # "V50",
    "V50_r0.5", 
   # "V60_r5.4", 
   # "V70_r0.4"
]

In [ ]:
profiles = load_profile.(["orbits/orbit1/1e6/"] .*  model_names);

In [ ]:
labels = Dict(
    "V32_r5" => "V=32, r=5.6", 
    "V32_r2.4" => "V=32, r=2.4",
    "V50" => "V=50, r=10.8",
    "V50_r0.5" => "V=50, r=5.4", 
    "V60_r5.4" => "V=60, r=5.4", 
    "V70_r0.4" => "V=70, r=6.5"
    )

labels = [labels[name] for name in model_names]

In [ ]:
let
	fig, ax = FigAxis(
		xlabel="time / Gyr",
		ylabel = L"$v_\textrm{circ, max}$ / km\,s$^{-1}$"
	)


	for i in eachindex(profiles)
		
		vs = [p.v_circ_max for p in profiles[i].profiles]
		lines!(profiles[i].times * T2GYR, (vs * V2KMS), label=labels[i])
	end

	axislegend()
	fig
end

In [ ]:
let
	fig, ax = FigAxis(
		xlabel="time / Gyr",
		ylabel = "bound number of particles",
	)

	for i in eachindex(profiles)
		
		vs = [p.N_bound for p in profiles[i].profiles]
		lines!(profiles[i].times * T2GYR, log10.(vs), label=labels[i])
	end

	axislegend()
	
	fig
end

In [ ]:
let
	fig, ax = FigAxis(
		xlabel=L"$\log\,r_\textrm{circ max}$ / kpc",
		ylabel = L"$\log\,v_\textrm{circ, max}$ / kpc",
	)

	for i in eachindex(profiles)
		profile = profiles[i]
		vs = [p.v_circ_max for p in profile.profiles]

		rs = [p.r_circ_max for p in profile.profiles]
		lines!(log10.(rs), log10.(vs * V2KMS), label=labels[i])
	end

	r = LinRange(1, 10, 100)
	v = 2π * r ./ (t_max_host /2)

	lines!(log10.(r), log10.(v * V2KMS), label=L"t=t_\textrm{max, host}/2", 
		color=:black, linestyle=:dot)

	axislegend(position=:rb)
	
	fig
end

In [ ]:
function t_max_of(prof)
	return 2π * prof.r_circ_max / prof.v_circ_max
end

In [ ]:
let
	fig, ax = FigAxis(
		xlabel="time / Gyr",
		ylabel = L"$t_\textrm{circ, max}$ / Gyr"
	)


	for i in eachindex(profiles)
		
		ts = t_max_of.(profiles[i].profiles)
		lines!(profiles[i].times * T2GYR, ts * T2GYR, label=labels[i])
	end

	hlines!(t_max_host * T2GYR / 2, color=:black, label=L"t_\textrm{max, host}/2")

	axislegend()
	fig
end

# Isolation

## Particle number

In [ ]:
model_names = ["isolation/"] .* [
    "1e4/fiducial",
    "1e5/fiducial",
    "1e6/fiducial",
    #"1e7"
    ]
labels = [
    "1e4",
    "1e5",
    "1e6",
    "1e7"
    ]

profiles = load_profile.(model_names);

In [ ]:
compare_densities(profiles, labels)

## 1e4

In [ ]:
print_cpus_in_folder("isolation/1e4")

In [ ]:
model_names = ["isolation/"] .* [
    "1e4/s0.014",
    "1e4/s0.44",
    "1e4/default",
    "1e4/s4"
    ]

labels = [
    "0.014",
    "0.44",
    "1.4",
    "4"
    ]

profiles = load_profile.(model_names);

In [ ]:
compare_densities(profiles, labels)

In [ ]:
model_names = ["isolation/"] .* [
    "1e4/dt_0.01",
    "1e4/dt_1",
    "1e4/dt_10",
    "1e4/default",
    ]

labels = model_names
profiles = load_profile.(model_names);

In [ ]:
compare_densities(profiles, labels)

In [ ]:
model_names = ["isolation/"] .* [
    "1e4/dt_0.01",
    "1e4/acc_0.001",
    "1e4/acc_0.003",
    "1e4/default",
    "1e4/acc_0.1",
    ]

labels = model_names
profiles = load_profile.(model_names);

In [ ]:
compare_densities(profiles, labels)

The above figure compares models using different relative timestepping criteria. The models perfectly converge around a parameter eta=0.003

### Gravitational force accuracy

In [ ]:
model_names = ["isolation/"] .* [
    "1e4/theta_0.1",
    "1e4/default",
    "1e4/theta_1",
    ]

labels = model_names
profiles = load_profile.(model_names);

In [ ]:
compare_densities(profiles, labels)

In [ ]:
model_names = ["isolation/"] .* [
    "1e4/theta_0.01",
    "1e4/theta_0.1",
    "1e4/default",
    "1e4/thetarel_0.5",
    ]

labels = model_names
profiles = load_profile.(model_names);

In [ ]:
compare_densities(profiles, labels)

The models above compare different values of the gravitational opening criterion. Below a theta of 0.1, the models are perfectly converged here. 

### Gravity methods

In [ ]:
model_names = ["isolation/1e4/"] .* [
    "dt_0.3",
    "treepm",
    "fmm",
    ]

labels = model_names
profiles = load_profile.(model_names);

In [ ]:
compare_densities(profiles, labels)

## 1e5

In [ ]:
print_cpus_in_folder("isolation/1e5")

In [ ]:
model_names = ["isolation/1e5/"] .* [
    "s1.4",
    "s0.44",
    "default",
   #"fiducial",
    "s0.044",
    "s0.014",
    ]

labels = model_names
profiles = load_profile.(model_names);

In [ ]:
compare_densities(profiles, labels)

In [ ]:
model_names = ["isolation/1e5/"] .* [
    "facc_0.015",
    "default",
   #"fiducial",
    "facc_0.001",
    ]

labels = model_names
profiles = load_profile.(model_names);

In [ ]:
compare_densities(profiles, labels)

## 1e6

In [ ]:
print_cpus_in_folder("1e6")

In [ ]:
profiles = [
    "s=0.014 kpc" => load_profile("1e6/s0.014"),
    "s=0.044 kpc" => load_profile("1e6/fiducial"),
    "s=0.14 kpc" => load_profile("1e6/s0.14"),
    ];

In [ ]:
compare_densities(profiles)

In [ ]:
compare_vcirc(profiles, errskip=10)

In [ ]:
compare_t_dyn(profiles)

## Halo generation methods

In [ ]:
profiles = [
    "agama" => load_profile("isolation/1e6/fiducial"),
    "gspmodel" => load_profile("isolation/1e6_gspmodel"),
    "gsp_realize" => load_profile("isolation/1e6_gsprealize"),
    ]

In [ ]:
compare_densities(profiles)

In [ ]:
compare_vcirc(profiles, labels)

In [ ]:
compare_vcirc1(profiles[1][end], res_limits=0.03)

In [ ]:
compare_vcirc1(profiles[2][end], NFWZeno(m_a=halo.M_s * LilGuys.A_NFW(1), a=halo.r_s, b=halo.r_s * 2))

In [ ]:
compare_vcirc1(profiles[3][end], NFWZeno(m_a=halo.M_s * LilGuys.A_NFW(1), a=halo.r_s, b=halo.r_s * 64), res_limits=0.03)

# 1e7

In [ ]:
print_cpus_in_folder("1e7/")

In [ ]:
profiles = [
    "s=0.014" => load_profile("1e7/fiducial"),
    #"s=0.014" => load_profile("isolation/1e7/s0.014")
    ];

In [ ]:
compare_densities(profiles,  errskip=200)

In [ ]:
compare_densities(profiles,  errskip=1)

In [ ]:
compare_vcirc(profiles, errskip=10)

In [ ]:
compare_t_dyn(profiles)